# Imports, variables, functions

In [17]:
# -- IMPORTS --

import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
import random
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from scipy import signal
from sklearn.ensemble import RandomForestClassifier

In [2]:
# -- ENVIRONMENT VARIABLES --

sample_rate = sampling_rate = 256
sec = 10
len_window = sample_rate*sec
overlap = 5
threshold = 2*sample_rate
sample_rate_downsample = int(0.1*sample_rate)
len_window_downsample = sample_rate_downsample*sec

patients = list(range(1, 80))
patient_with_issue = [4, 29, 50] 
for i in patient_with_issue:
    patients.remove(i)

# Load annotation file
annt = scipy.io.loadmat('../raw_data/annotations_2017.mat')

In [3]:
## -- PREPROCESSING FUNCTIONS --

# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

# Create our own scaler
class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

# Combination of all filters and Scaler
def filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(filter_hp, sampling_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

In [4]:
## -- LABEL FUNCTIONS --

# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C', 'ECG EKG', 'Resp Effort'], inplace=True)
    return df

# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    eeg_patient.rename(columns={'ECG EKG-REF':'ECG EKG', 'Resp Effort-REF':'Resp Effort'}, inplace=True)
    
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

In [5]:
## -- FEATURE ENGINEERING --

def flatten(window_df):
    if len(np.unique(window_df.iloc[:,-1])) == 1:
        target = window_df.iloc[0,-1]
    elif np.unique(window_df.iloc[:,-1],return_counts=True)[1][1] >= threshold:
        target = 1
    else:
        target = 0
    t_df = window_df.drop(columns = "target").transpose()
    flatten = pd.DataFrame(np.array(t_df).reshape(1,t_df.shape[0]*t_df.shape[1]))
    flatten["Target"] = target
    return flatten

In [6]:
## -- MAIN FUNCTIONS --

def preprocess_and_label(path_raw_data, scaler, patient_number, Fournier=False):
    
    # Load raw data
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    # Preprocess data 
    signals_preprocessed = filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    
    if Fournier == True:
        signals_preprocessed = pd.DataFrame(np.array([abs(rfft(signals_preprocessed[i])) for i in range(len(signals_preprocessed))]))
        
    # Label data
    df = label_data(path_raw_data, signals_preprocessed, patient_number)
    
    return df

def downsampling(df):
    df_downsample = pd.DataFrame()
    all_df = pd.DataFrame()
    target_col = df.iloc[:,-1]
    num = int(0.1*sample_rate)
    t = 256
    for i, column in enumerate(df.columns[:-1]):
        df_downsample = pd.DataFrame()
        for j in range(0,len(df)-sample_rate,sample_rate):
            x = np.array(df.iloc[j:j+sample_rate,i])
            x_resampled = pd.DataFrame(signal.resample(x, num), index=range(int(j/10),int(j/10)+num))
            df_downsample= pd.concat([df_downsample,x_resampled])
        all_df[column] = np.array(df_downsample).reshape(1,-1)[0]
    target = []
    for t in range(0,len(target_col)-256,sample_rate):
        for n in range(num):
            target.append(target_col[t])
    all_df["target"] = target
    return all_df

def flatten_dataframe(df):
    data = np.array([flatten(df.iloc[i:i+len_window_downsample]) for i in range(0,len(df)-len_window_downsample, overlap*sample_rate_downsample)])
    r=data.shape[0]
    c=data.shape[2]
    data = pd.DataFrame(data.reshape(r,c))
    return data

def concat_dataframes(dictionnary_of_dataframes):
    df = pd.concat([dictionnary_of_dataframes[i] for i in dictionnary_of_dataframes.keys()])
    return df

def create_x_and_y(df):
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X,y

def oversampling(X, y): 
    sm = SMOTE(sampling_strategy='minority', random_state=7)
    X, y = sm.fit_resample(X, y)
    return X, y

# EEGs of unhealthy patients

In [7]:
patients = list(range(1, 80))

patient_with_issue = [4, 29, 50] # Can't import ECG4, ECG29 and ECG50
patient_without_seizure = [2, 3, 6, 8, 10, 12, 18, 24, 26, 27, 28, 30, 32, 35, 37, 42, 43, 45, 46, 48, 49, 53, 55, 56, 57, 58, 59, 60, 61,65, 70, 72]

for i in patient_with_issue:
    patients.remove(i)

for i in patient_without_seizure:
    patients.remove(i)

# Cross-validation for each patient

In [8]:
def model_for_each_patient(i, model):
    # Preprocess and label
    df_i = preprocess_and_label(f"../raw_data/eeg{i}.edf", CustomTranformer(), i, Fournier=False)
    
    # Downsample
    df_i_downsample = downsampling(df_i)
    
    # Flatten
    df_i_flat = flatten_dataframe(df_i_downsample)

    # Create x and y
    X, y = create_x_and_y(df_i_flat)
    
    # Cross validation    
    cv_results = cross_validate(model, X, y, cv=5, scoring=['accuracy', 'recall', 'f1'])
    
    # Mean of scores
    accuracy = cv_results['test_accuracy'].mean()
    recall = cv_results['test_recall'].mean()
    f1 = cv_results['test_f1'].mean()
    
    return accuracy, recall, f1

In [9]:
model = SVC(kernel='linear', C=0.001) 

accuracy_scores = []
recall_scores = []
f1_scores = []

for i in patients:
    accuracy, recall, f1 = model_for_each_patient(i, model)
    accuracy_scores.append(accuracy)
    recall_scores.append(recall)
    f1_scores.append(f1)

/Users/florencetersier/.pyenv/versions/3.10.6/envs/SeizurePredict/lib/python3.10/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/florencetersier/.pyenv/versions/3.10.6/envs/SeizurePredict/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/florencetersier/.pyenv/versions/3.10.6/envs/SeizurePredict/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1599: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [12]:
results = pd.DataFrame(accuracy_scores, patients)
results.rename(columns={0:'accuracy'}, inplace=True)
results['recall'] = recall_scores
results['f1_score'] = f1_scores
results

,accuracy,recall,f1_score
1,0.664199,0.203922,0.187119
5,0.688057,0.793413,0.807902
7,0.758573,0.205138,0.197595
9,0.699131,0.151261,0.196469
11,0.985958,0.066667,0.100000
13,0.905542,0.189260,0.253946
14,0.537149,0.387179,0.466409
15,0.752516,0.164444,0.173089
16,0.758043,0.170210,0.198187
17,0.958078,0.057143,0.053333


In [13]:
df_train = pd.read_csv("../data_modeling/df_train.csv")

In [22]:
df_test = pd.read_csv("../data_modeling/df_test.csv")

In [14]:
df_train.shape

(52779, 4753)

In [15]:
X_train, y_train = create_x_and_y(df_train)

In [23]:
X_test, y_test = create_x_and_y(df_test)

In [18]:
model = RandomForestClassifier()

In [19]:
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
results_random

In [25]:
# Train accuracy
y_pred_train = model.predict(X_train)

train_score = accuracy_score(y_train, y_pred_train)
train_score

0.9999621061406999

In [24]:
# Test accuracy
y_pred_test = model.predict(X_test)

test_score = accuracy_score(y_test, y_pred_test)
test_score

0.8931918656056587

In [26]:
# Preprocess and label new data
path_raw_new_data = "../raw_data/eeg5.edf"
df_new = preprocess_and_label(path_raw_new_data, CustomTranformer(), 5, Fournier=False)

# Downsampling new data
df_new_downsample = downsampling(df_new)

# Flatten new data
df_new_downsample_flat = flatten_dataframe(df_new_downsample)

In [33]:
X_train.shape

(52779, 4752)

In [32]:
df_new_downsample_flat.shape

(766, 4751)

In [31]:
df_new_downsample_flat.iloc[:,:-1]

,0,1,2,3,4,5,6,7,8,9,...,4740,4741,4742,4743,4744,4745,4746,4747,4748,4749
0,18.465606,-26.306646,-32.302281,-10.235163,-21.022621,-8.148782,-16.475783,-11.992139,2.222471,-19.619645,...,25.765433,27.815653,-2.003792,3.118333,9.023849,-13.086006,-19.783280,-4.977995,7.521648,-7.660524
1,14.341851,72.583393,40.512054,31.137797,30.771191,40.940802,36.437042,23.757499,-7.975942,5.179819,...,47.151492,10.406856,-20.720155,-8.643154,15.467842,24.181954,24.032155,24.468062,9.462268,15.379352
2,-18.897338,4.864978,13.298124,-5.448292,-0.646112,-17.118978,-11.438369,2.880844,7.860662,-14.092426,...,-2.611275,9.594366,2.780385,14.047313,25.527503,22.167088,6.573288,-21.209199,-7.460495,12.379000
3,4.787344,-7.341195,-7.052898,-17.708414,-10.006758,-7.390754,-23.280595,2.305803,41.031039,64.431358,...,60.269986,7.588003,-19.346224,1.596917,27.314869,29.200332,32.044138,32.098210,5.600330,-26.769344
4,9.513777,3.717521,14.537376,-6.459300,-4.850454,-15.903774,-23.118770,-29.961382,-14.154079,0.034269,...,-4.881568,12.361830,19.417275,11.082555,17.360553,19.231238,12.106461,14.248537,-2.603800,-9.085924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,72.851485,-42.370209,-18.550089,0.718367,18.376838,16.555561,27.116725,15.830382,52.270031,42.843238,...,3.689148,-4.933979,-2.878427,6.063524,13.190644,-20.408805,-43.095636,-32.206555,-12.255806,-15.595251
762,-12.056854,-15.495535,-17.642040,-18.758064,-52.783004,-35.626593,-34.448700,-38.117326,-47.276862,-47.336752,...,-8.322157,-32.349122,-67.390159,-79.255403,-61.429528,-44.037072,-38.125400,-14.000050,-10.324536,0.809076
763,26.641879,21.857497,23.675274,1.950207,17.491731,19.506996,10.162714,7.255041,15.156347,0.592630,...,17.922081,10.114713,27.686684,38.577015,35.386873,22.962240,15.317349,-12.095352,-8.851452,16.624332
764,-16.194946,-15.903980,-32.216809,-21.519892,-10.978835,-14.637629,-6.993414,-7.433200,4.158676,4.053450,...,-5.495766,-10.147151,-5.388581,-17.070011,-5.369319,-4.607522,-8.283338,-8.243578,-0.727881,13.791996


In [29]:
# Accuracy
y_true = df_new_downsample_flat.iloc[:,-1]
y_pred = model.predict(df_new_downsample_flat.iloc[:,:-1])

new_score = accuracy_score(y_true, y_pred)
new_score

/Users/florencetersier/.pyenv/versions/3.10.6/envs/SeizurePredict/lib/python3.10/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 4750 features, but RandomForestClassifier is expecting 4752 features as input.